**Speech To Text Uygulaması**


---
Yazılacak bu uygulamanın amacı, mevcut bir ses dosyasını veya gerçek zamanlı konuşmaları yazıya çevirmektir.

Bu uygulama içerisinde, Python yazılım dili kullanılacak olup, "***DeepSpeech***" kütüphanesinden yararlanılacaktır. DeepSpeech kütüphanesi, açık kaynak kodlu makine öğrenmesine bağlı olarak geliştirilmiş bir speech to text (STT) kütühanesidir. DeepSpeech temelinde Google tarafından geliştirilen ve makine öğrenmesinde kullanılan "***TensorFlow***" kütüphanesine dayanmaktadır. TensorFlow verilen dataset'lere (video, ses, resim vb.) göre bir model oluşturmaktır. DeepSpeech TensorFlow ile oluşturulan bu modellere dayanarak istenilen sesi yazıya çevirir. 


**1.   Gerekli Kütüphanelerin Kurulması**

Aşağıda bulunan komutlar ile DeepSpeech kütüphanesi ve model yüklenir.


In [ ]:
#DeepSpeech yuklenmesi
!pip3 install deepspeech

In [ ]:
#modellin indirilmesi
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer


In [ ]:
#ornek ses dosyalarinin indirilmesi
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/audio-0.9.3.tar.gz
!tar xvf audio-0.9.3.tar.gz
!ls -ls ./audio

In [ ]:
!ls -la


**2.   Kodun Yazilmasi**

Aşağıda yer alan kodlarda DeepSpeech kütüphanesi kullanılarak verilen bir ses dosyasının yazıya dönüştürmesini yapan kod blokları yer almaktadır.

In [5]:
#ilgili kütüphanelerin eklenmesi
from deepspeech import Model
import numpy as np
import os
import wave 
from IPython.display import Audio

In [6]:
"""DeepSpeech kütüpanesine gore hangi model ve 
hangi band aralıklarında dönüştürme yapacağını gösteren kod blokları"""
model_file_path = 'deepspeech-0.9.3-models.pbmm'
lm_file_path = 'deepspeech-0.9.3-models.scorer'
beam_width = 100
lm_alpha = 0.93
lm_beta = 1.18
model = Model(model_file_path)
model.enableExternalScorer(lm_file_path)
model.setScorerAlphaBeta(lm_alpha, lm_beta)
model.setBeamWidth(beam_width)

In [8]:
#verilen ses dosyasının frame, rate ve buffer değerlerini döndüren fonksiyon
def read_wav_file(filename):
  with wave.open(filename, 'rb') as w:
    rate = w.getframerate()
    frames = w.getnframes()
    buffer = w.readframes(frames)
    print("Rate:", rate)
    print("Frames:",frames)
    print("Buffer Len:", len(buffer))
  return buffer, rate

In [9]:
#buffer, rate ve frame değerine göre ilgili sesi yazıya doker.
def transcribe_batch(audio_file):
    buffer, rate = read_wav_file(audio_file)
    data16 = np.frombuffer(buffer, dtype=np.int16)
    return model.stt(data16)

In [10]:
#ses dosyasının dinlenmesi
Audio('audio/2830-3980-0043.wav')

In [11]:
# ses dosyasını yazıya aktaran kod blogu
transcribe_batch('audio/2830-3980-0043.wav')

Rate: 16000
Frames: 31600
Buffer Len: 63200


'experience proves this'

In [12]:
# bir ses dosyası DeepSpeech tarafından okunurken mevcut framedeyken yazıya aktarması için stram modeli.
stream = model.createStream()

In [13]:
#stream modeline göre verilen buffer, rate ve frame'e gore sesi yazıya aktaran fonksiyon
from IPython.display import clear_output
def transcribe_streaming(audio_file):
    buffer, rate = read_wav_file(audio_file)
    offset=0
    batch_size=8196
    text=""

    while offset < len(buffer):
      end_offset=offset+batch_size
      chunk=buffer[offset:end_offset]
      data16 = np.frombuffer(chunk, dtype=np.int16)

      stream.feedAudioContent(data16)
      text=stream.intermediateDecode()
      clear_output(wait=True)
      print(text)
      offset=end_offset
    return True